In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 評估代理 - 使用 Vertex AI Gen AI 評估服務評估 LangGraph 代理

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> 在 Colab 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fevaluation%2Fevaluating_langgraph_agent.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> 在 Colab Enterprise 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/evaluation/evaluating_langgraph_agent.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> 在 GitHub 上檢視
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>分享至：</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/evaluation/evaluating_langgraph_agent.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| 作者 | [Ivan Nardini](https://github.com/inardini) [Naveksha Sood](https://github.com/navekshasood)|

## 總覽

與任何生成式 AI 應用程式一樣，AI 代理需要徹底的評估，以確保它們可靠且有效地執行。這種評估應該在即時 (線上) 和大型測試案例資料集 (離線) 上進行。建構代理應用程式的開發人員在評估其效能方面面臨重大挑戰。主觀 (人類回饋) 和客觀 (可衡量指標) 評估對於建立對代理行為的信任至關重要。

Vertex AI 模型評估提供了一個品質受控且可解釋的方法和指標工具包，可用於評估任何生成式模型或應用程式 (包括代理)，並使用您自己的評估標準將評估結果與您自己的判斷進行基準比較。

本教學示範如何使用 Vertex AI Gen AI 評估來評估 LangGraph 代理。

本教學使用以下 Google Cloud 服務和資源：

*  Vertex AI Gen AI 評估

執行的步驟包括：

* 使用 LangGraph 建構本地代理
* 準備代理評估資料集
* 單一工具使用評估
* 軌跡評估
* 回應評估


## 開始

### 安裝 Vertex AI SDK 和其他必要套件


In [ ]:
%pip install "langchain_google_vertexai" "langgraph"
%pip install --upgrade --user --quiet "google-cloud-aiplatform[evaluation]"

### 重新啟動執行階段

要在這個 Jupyter 執行階段中使用新安裝的套件，您必須重新啟動執行階段。您可以透過執行下面的儲存格來執行此操作，它會重新啟動目前的內核。

重新啟動可能需要一分鐘或更長時間。重新啟動後，繼續下一步。

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 內核即將重新啟動。在 Colab 或 Colab Enterprise 中，您可能會看到一條錯誤訊息，指出「您的會話因不明原因崩潰」。這是預期行為。請等到它完成後再繼續下一步。⚠️</b>
</div>


### 驗證您的筆記本環境 (僅限 Colab)

如果您在 Google Colab 上執行此筆記本，請執行以下儲存格以驗證您的環境。

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### 設定 Google Cloud 專案資訊並初始化 Vertex AI SDK

要開始使用 Vertex AI，您必須擁有一個現有的 Google Cloud 專案並[啟用 Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com)。

了解更多關於[設定專案和開發環境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)的資訊。

In [ ]:
# 如果使用者未提供專案 ID，則使用環境變數。
import os

import vertexai

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}

if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

EXPERIMENT_NAME = "evaluate-langgraph-agent"  # @param {type:"string"}

vertexai.init(project=PROJECT_ID, location=LOCATION, experiment=EXPERIMENT_NAME)

## 匯入函式庫

匯入教學所需的函式庫。

In [ ]:
import json

# 一般
import random
import string
from typing import Literal

from IPython.display import HTML, Markdown, display

# 評估代理
from google.cloud import aiplatform
from langchain.load import dump as langchain_load_dump

# 建構代理
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessageGraph
from langgraph.prebuilt import ToolNode
import pandas as pd
import plotly.graph_objects as go
from vertexai.preview.evaluation import EvalTask
from vertexai.preview.evaluation.metrics import (
    PointwiseMetric,
    PointwiseMetricPromptTemplate,
    TrajectorySingleToolUse,
)

## 定義輔助函式

初始化一組輔助函式以列印教學結果。

In [ ]:
def get_id(length: int = 8) -> str:
    """產生指定長度的 uuid (預設=8)。"""
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


def parse_messages_to_output_dictionary(messages: list[dict]) -> dict:
    """將建構函式格式的訊息列表解析為回應和函式呼叫。"""

    final_output = {
        "response": "在訊息歷史記錄中找不到 AI 回應。",
        "predicted_trajectory": [],
    }

    # 處理每則訊息
    function_calls = []
    for message in messages:
        # 檢查它是否是包含實際回應的工具訊息
        if message.get("type") == "constructor" and "ToolMessage" in message.get(
            "id", []
        ):
            final_output["response"] = message["kwargs"]["content"]

        # 檢查它是否是 AI 訊息以獲取工具呼叫
        elif message.get("type") == "constructor" and "AIMessage" in message.get(
            "id", []
        ):
            tool_calls = message["kwargs"].get("tool_calls", [])
            for tool_call in tool_calls:
                if tool_call:
                    function_calls.append(
                        {
                            "tool_name": tool_call.get("name"),
                            "tool_input": tool_call.get("args"),
                        }
                    )

    final_output["predicted_trajectory"] = json.dumps(function_calls)
    return final_output


def format_output_as_markdown(output: dict) -> str:
    """將輸出字典轉換為格式化的 markdown 字串。"""
    markdown = "### AI 回應\n"
    markdown += f"{output['response']}\n\n"

    if output["predicted_trajectory"]:
        output["predicted_trajectory"] = json.loads(output["predicted_trajectory"])
        markdown += "### 函式呼叫\n"
        for call in output["predicted_trajectory"]:
            markdown += f"- **函式**: `{call['tool_name']}`\n"
            markdown += "  - **參數**:\n"
            for key, value in call["tool_input"].items():
                markdown += f"    - `{key}`: `{value}`\n"

    return markdown


def display_eval_report(eval_result: pd.DataFrame) -> None:
    """顯示評估結果。"""
    metrics_df = pd.DataFrame.from_dict(eval_result.summary_metrics, orient="index").T
    display(Markdown("### 摘要指標"))
    display(metrics_df)

    display(Markdown(f"### 逐行指標"))
    display(eval_result.metrics_table)


def display_drilldown(row: pd.Series) -> None:
    """顯示列中軌跡資料的深入檢視。"""

    style = "white-space: pre-wrap; width: 800px; overflow-x: auto;"

    if not (
        isinstance(row["predicted_trajectory"], list)
        and isinstance(row["reference_trajectory"], list)
    ):
        return

    for predicted_trajectory, reference_trajectory in zip(
        row["predicted_trajectory"], row["reference_trajectory"]
    ):
        display(
            HTML(
                f"<h3>工具名稱:</h3><div style='{style}'>{predicted_trajectory['tool_name'], reference_trajectory['tool_name']}</div>"
            )
        )

        if not (
            isinstance(predicted_trajectory.get("tool_input"), dict)
            and isinstance(reference_trajectory.get("tool_input"), dict)
        ):
            continue

        for tool_input_key in predicted_trajectory["tool_input"]:
            print("工具輸入鍵: ", tool_input_key)

            if tool_input_key in reference_trajectory["tool_input"]:
                print(
                    "工具值: ",
                    predicted_trajectory["tool_input"][tool_input_key],
                    reference_trajectory["tool_input"][tool_input_key],
                )
            else:
                print(
                    "工具值: ",
                    predicted_trajectory["tool_input"][tool_input_key],
                    "N/A",
                )
        print("\n")
    display(HTML("<hr>"))


def display_dataframe_rows(
    df: pd.DataFrame,
    columns: list[str] | None = None,
    num_rows: int = 3,
    display_drilldown: bool = False,
) -> None:
    """顯示 DataFrame 的一部分行，可選擇性地包含深入檢視。"""

    if columns:
        df = df[columns]

    base_style = "font-family: monospace; font-size: 14px; white-space: pre-wrap; width: auto; overflow-x: auto;"
    header_style = base_style + "font-weight: bold;"

    for _, row in df.head(num_rows).iterrows():
        for column in df.columns:
            display(
                HTML(
                    f"<span style='{header_style}'>{column.replace('_', ' ').title()}: </span>"
                )
            )
            display(HTML(f"<span style='{base_style}'>{row[column]}</span><br>"))

        display(HTML("<hr>"))

        if (
            display_drilldown
            and "predicted_trajectory" in df.columns
            and "reference_trajectory" in df.columns
        ):
            display_drilldown(row)


def plot_bar_plot(
    eval_result: pd.DataFrame, title: str, metrics: list[str] = None
) -> None:
    fig = go.Figure()
    data = []

    summary_metrics = eval_result.summary_metrics
    if metrics:
        summary_metrics = {
            k: summary_metrics[k]
            for k, v in summary_metrics.items()
            if any(selected_metric in k for selected_metric in metrics)
        }

    data.append(
        go.Bar(
            x=list(summary_metrics.keys()),
            y=list(summary_metrics.values()),
            name=title,
        )
    )

    fig = go.Figure(data=data)

    # 變更長條圖模式
    fig.update_layout(barmode="group")
    fig.show()


def display_radar_plot(eval_results, title: str, metrics=None):
    """繪製雷達圖。"""
    fig = go.Figure()
    summary_metrics = eval_results.summary_metrics
    if metrics:
        summary_metrics = {
            k: summary_metrics[k]
            for k, v in summary_metrics.items()
            if any(selected_metric in k for selected_metric in metrics)
        }

    min_val = min(summary_metrics.values())
    max_val = max(summary_metrics.values())

    fig.add_trace(
        go.Scatterpolar(
            r=list(summary_metrics.values()),
            theta=list(summary_metrics.keys()),
            fill="toself",
            name=title,
        )
    )
    fig.update_layout(
        title=title,
        polar=dict(radialaxis=dict(visible=True, range=[min_val, max_val])),
        showlegend=True,
    )
    fig.show()

## 建構 LangGraph 代理

使用 LangGraph 建構您的應用程式，包括 Gemini 模型、您定義的自訂工具以及用於控制對話流程的路由器。

### 設定工具

首先，設定客戶支援代理完成工作所需的工具。

In [ ]:
@tool
def get_product_details(product_name: str):
    """收集產品的基本詳細資訊。"""
    details = {
        "smartphone": "一款具有先進相機功能和閃電般處理速度的尖端智慧型手機。",
        "usb charger": "一款超快、輕便的 USB 充電器",
        "shoes": "專為舒適、支撐和速度而設計的高性能跑鞋。",
        "headphones": "具有先進降噪技術的無線耳機，可提供身臨其境的音訊體驗。",
        "speaker": "一款聲控智慧喇叭，可播放音樂、設定鬧鐘並控制智慧家居設備。",
    }
    return details.get(product_name, "找不到產品詳細資訊。")


@tool
def get_product_price(product_name: str):
    """收集產品的價格資訊。"""
    details = {
        "smartphone": 500,
        "usb charger": 10,
        "shoes": 100,
        "headphones": 50,
        "speaker": 80,
    }
    return details.get(product_name, "找不到產品價格。")

### 定義路由器

設定一個路由器，透過根據使用者輸入或互動狀態選擇適當的工具來引導對話流程。


In [ ]:
def router(
    state: list[BaseMessage],
) -> Literal["get_product_details", "get_product_price", END]:
    """如果使用者詢問產品，則啟動產品詳細資訊或價格檢索。"""
    # 從對話歷史記錄的最後一則訊息中獲取 tool_calls。
    tool_calls = state[-1].tool_calls

    # 如果有任何 tool_calls
    if tool_calls:
        # 檢查第一個工具呼叫中的函式名稱
        function_name = tool_calls[0].get("name")
        if function_name == "get_product_price":
            return "get_product_price"
        else:
            return "get_product_details"
    else:
        # 結束對話流程。
        return END

### 設定模型

選擇您的代理將使用的 Gemini AI 模型。如果您對 Gemini 及其不同功能感到好奇，請查看[官方文件](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models)以獲取更多詳細資訊。

In [ ]:
llm = "gemini-2.0-flash"

### 組裝代理

Vertex AI Gen AI 評估可直接與「可查詢 (Queryable)」的代理一起使用，也允許您使用特定結構 (簽章) 新增自己的自訂函式。

在這種情況下，您使用自訂函式來組裝代理。該函式會針對給定的輸入觸發代理，並解析代理的結果以提取回應和被呼叫的工具。

In [ ]:
def agent_parsed_outcome(input):

    model = ChatVertexAI(model=llm)
    builder = MessageGraph()

    model_with_tools = model.bind_tools([get_product_details, get_product_price])
    builder.add_node("tools", model_with_tools)

    tool_node = ToolNode([get_product_details, get_product_price])
    builder.add_node("get_product_details", tool_node)
    builder.add_node("get_product_price", tool_node)
    builder.add_edge("get_product_details", END)
    builder.add_edge("get_product_price", END)

    builder.set_entry_point("tools")
    builder.add_conditional_edges("tools", router)

    app = builder.compile()
    chat_history = langchain_load_dump.dumpd(app.invoke(HumanMessage(input)))
    return parse_messages_to_output_dictionary(chat_history)

### 測試代理

查詢您的代理。

In [ ]:
response = agent_parsed_outcome(input="獲取鞋子的產品詳細資訊")
display(Markdown(format_output_as_markdown(response)))

In [ ]:
response = agent_parsed_outcome(input="獲取鞋子的產品價格")
display(Markdown(format_output_as_markdown(response)))

## 使用 Vertex AI Gen AI 評估來評估 LangGraph 代理

在處理 AI 代理時，追蹤其效能以及它們的運作情況非常重要。您可以從兩個主要方面來看待這個問題：**監控**和**可觀測性**。

監控專注於您的代理在執行特定任務時的表現：

* **單一工具選擇**：代理是否為工作選擇了正確的工具？

* **多個工具選擇 (或軌跡)**：代理在使用工具的順序上是否做出合乎邏輯的選擇？

* **回應生成**：代理的輸出是否良好，並且根據其使用的工具是否有意義？

可觀測性是關於了解代理的整體健康狀況：

* **延遲**：代理回應需要多長時間？

* **失敗率**：代理無法產生回應的頻率是多少？

Vertex AI Gen AI 評估服務可幫助您在原型設計階段或將代理部署到生產環境後評估所有這些方面。它提供[預建的評估標準和指標](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval)，因此您可以確切地了解您的代理的表現，並找出需要改進的地方。

### 準備代理評估資料集

要使用 Vertex AI Gen AI 評估服務評估您的 AI 代理，您需要一個特定的資料集，具體取決於您想要評估代理的哪些方面。

此資料集應包括提供給代理的提示。它還可以包含理想或預期的回應 (真實情況) 以及代理應採取的預期工具呼叫序列 (參考軌跡)，代表您期望代理針對每個給定提示呼叫的工具序列。

> 可選地，您可以提供生成的回應和預測的軌跡 (**自備資料集情境**)。

以下是您可能擁有的客戶支援代理的資料集範例，其中包含使用者提示和參考軌跡。

In [ ]:
eval_data = {
    "prompt": [
        "獲取智慧型手機的價格",
        "獲取耳機的產品詳細資訊和價格",
        "獲取 USB 充電器的詳細資訊",
        "獲取鞋子的產品詳細資訊和價格",
        "獲取喇叭的產品詳細資訊？",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "smartphone"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "headphones"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "headphones"},
            },
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "usb charger"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "shoes"},
            },
            {"tool_name": "get_product_price", "tool_input": {"product_name": "shoes"}},
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "speaker"},
            }
        ],
    ],
}

eval_sample_dataset = pd.DataFrame(eval_data)

列印資料集中的一些範例。

In [ ]:
display_dataframe_rows(eval_sample_dataset, num_rows=3)

### 單一工具使用評估

在您設定好 AI 代理和評估資料集後，您可以開始評估代理是否為給定任務選擇了正確的單一工具。


#### 設定單一工具使用指標

Vertex AI Gen AI 評估中的 `trajectory_single_tool_use` 指標提供了一種快速的方法來評估您的代理是否使用了您期望它使用的工具，而不管任何特定的工具順序。這是一個基本但有用的方法，可以開始評估在代理的過程中是否在某個點上使用了正確的工具。

要使用 `trajectory_single_tool_use` 指標，您需要設定對於特定使用者的請求應該使用哪個工具。例如，如果使用者要求「傳送電子郵件」，您可能會期望代理使用「send_email」工具，並且在使用此指標時您會指定該工具的名稱。


In [ ]:
single_tool_usage_metrics = [TrajectorySingleToolUse(tool_name="get_product_price")]

#### 執行評估任務

要執行評估，您需要在一個實驗中，使用預先定義的資料集 (`eval_sample_dataset`) 和指標 (`single_tool_usage_metrics` 在此案例中) 來初始化一個 `EvalTask`。然後，您使用 `agent_parsed_outcome` 函式執行評估，並為此特定的評估執行指派一個唯一的識別碼，儲存和視覺化評估結果。


In [ ]:
EXPERIMENT_RUN = f"single-metric-eval-{get_id()}"

single_tool_call_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=single_tool_usage_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/single-metric-eval",
)

single_tool_call_eval_result = single_tool_call_eval_task.evaluate(
    runnable=agent_parsed_outcome, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(single_tool_call_eval_result)

#### 視覺化評估結果

使用一些輔助函式來視覺化評估結果的範例。

In [ ]:
display_dataframe_rows(single_tool_call_eval_result.metrics_table, num_rows=3)

### 軌跡評估

在評估代理為給定任務選擇單一最合適工具的能力之後，您可以透過分析相對於使用者輸入的工具序列選擇 (軌跡) 來概括評估。這將評估代理是否不僅選擇了正確的工具，而且還以合理且有效的方式使用它們。

#### 設定軌跡指標

要評估代理的軌跡，Vertex AI Gen AI 評估提供了幾個基於真實情況的指標：

* `trajectory_exact_match`：完全相同的軌跡 (相同的動作，相同的順序)

* `trajectory_in_order_match`：參考動作按順序出現在預測軌跡中 (允許額外動作)

* `trajectory_any_order_match`：所有參考動作都出現在預測軌跡中 (順序、額外動作不重要)。

* `trajectory_precision`：預測動作中出現在參考中的比例

* `trajectory_recall`：參考動作中出現在預測中的比例。

所有指標的得分為 0 或 1，除了 `trajectory_precision` 和 `trajectory_recall` 的範圍是 0 到 1。

In [ ]:
trajectory_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "trajectory_any_order_match",
    "trajectory_precision",
    "trajectory_recall",
]

#### 執行評估任務

透過執行新 `EvalTask` 的 `evaluate` 方法來提交評估。

In [ ]:
EXPERIMENT_RUN = f"trajectory-{get_id()}"

trajectory_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=trajectory_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/multiple-metric-eval",
)

trajectory_eval_result = trajectory_eval_task.evaluate(
    runnable=agent_parsed_outcome, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(trajectory_eval_result)

#### 視覺化評估結果

列印並視覺化評估結果的範例。

In [ ]:
display_dataframe_rows(trajectory_eval_result.metrics_table, num_rows=3)

In [ ]:
plot_bar_plot(
    trajectory_eval_result,
    title="軌跡指標",
    metrics=[f"{metric}/mean" for metric in trajectory_metrics],
)

### 評估最終回應

與模型評估類似，您可以使用 Vertex AI Gen AI 評估來評估代理的最終回應。

#### 設定回應指標

在代理推斷之後，Vertex AI Gen AI 評估提供了幾個指標來評估生成的回應。您可以使用基於計算的指標將回應與參考進行比較 (如果需要)，並使用現有或自訂的基於模型的指標來確定最終回應的品質。

查看[文件](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval)以了解更多資訊。


In [ ]:
response_metrics = ["safety", "coherence"]

#### 執行評估任務

要評估代理生成的回應，請使用 EvalTask 類別的 `evaluate` 方法。

In [ ]:
EXPERIMENT_RUN = f"response-{get_id()}"

response_eval_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/response-metric-eval",
)

response_eval_result = response_eval_task.evaluate(
    runnable=agent_parsed_outcome, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(response_eval_result)

#### 視覺化評估結果


列印新的評估結果範例。

In [ ]:
display_dataframe_rows(response_eval_result.metrics_table, num_rows=3)

### 根據工具選擇評估生成的回應

在評估與環境互動的 AI 代理時，標準的文字生成指標 (如連貫性) 可能不夠。這是因為這些指標主要關注文字結構，而代理的回應應根據其在環境中的有效性進行評估。

相反地，應使用自訂指標來評估代理的回應是否從其工具選擇中邏輯地得出，就像您在本節中看到的一樣。

#### 定義一個自訂指標

根據[文件](https://cloud.google.com/vertex-ai/generative-ai/docs/models/determine-eval#model-based-metrics)，您可以定義一個提示模板，透過設定評估標準和評分系統來評估 AI 代理的回應是否從其行動中邏輯地得出。

定義一個 `criteria` 來設定評估指南，並定義一個 `pointwise_rating_rubric` 來提供評分系統 (1 或 0)。然後使用 `PointwiseMetricPromptTemplate` 來使用這些元件建立模板。


In [ ]:
criteria = {
    "遵循軌跡": (
        "評估代理的回應是否從其採取的行動序列中邏輯地得出。"
        "考慮以下幾點：\n"
        "  - 回應是否反映了在軌跡中收集到的資訊？\n"
        "  - 回應是否與任務的目標和約束一致？\n"
        "  - 推理中是否有任何意想不到或不合邏輯的跳躍？\n"
        "提供來自軌跡和回應的具體範例以支持您的評估。"
    )
}

pointwise_rating_rubric = {
    "1": "遵循軌跡",
    "0": "不遵循軌跡",
}

response_follows_trajectory_prompt_template = PointwiseMetricPromptTemplate(
    criteria=criteria,
    rating_rubric=pointwise_rating_rubric,
    input_variables=["prompt", "predicted_trajectory"],
)

列印此模板的 prompt_data，其中包含組合的標準和評分標準資訊，可用於評估。

In [ ]:
print(response_follows_trajectory_prompt_template.prompt_data)

在您定義了評估提示模板之後，設定相關的指標來評估一個回應遵循特定軌跡的程度。`PointwiseMetric` 建立一個指標，其中 `response_follows_trajectory` 是指標的名稱，而 `response_follows_trajectory_prompt_template` 提供了您之前設定的評估說明或上下文。


In [ ]:
response_follows_trajectory_metric = PointwiseMetric(
    metric="response_follows_trajectory",
    metric_prompt_template=response_follows_trajectory_prompt_template,
)

#### 設定回應指標

透過包含自訂指標來設定新的生成回應評估指標。


In [ ]:
response_tool_metrics = [
    "trajectory_exact_match",
    "trajectory_in_order_match",
    "safety",
    response_follows_trajectory_metric,
]

#### 執行評估任務

執行新的代理評估。

In [ ]:
EXPERIMENT_RUN = f"response-over-tools-{get_id()}"

response_eval_tool_task = EvalTask(
    dataset=eval_sample_dataset,
    metrics=response_tool_metrics,
    experiment=EXPERIMENT_NAME,
    output_uri_prefix=BUCKET_URI + "/reasoning-metric-eval",
)

response_eval_tool_result = response_eval_tool_task.evaluate(
    runnable=agent_parsed_outcome, experiment_run_name=EXPERIMENT_RUN
)

display_eval_report(response_eval_tool_result)

#### 視覺化評估結果

視覺化評估結果範例。

In [ ]:
display_dataframe_rows(response_eval_tool_result.metrics_table, num_rows=3)

## 額外加分：自備資料集 (BYOD) 並使用 Vertex AI Gen AI 評估來評估 LangGraph 代理

在自備資料集 (BYOD) [情境](https://cloud.google.com/vertex-ai/generative-ai/docs/models/evaluation-dataset)中，您需要提供代理的預測軌跡和生成的回應。


### 自備您的評估資料集

使用預測的軌跡和生成的回應來定義評估資料集。

In [ ]:
byod_eval_data = {
    "prompt": [
        "獲取智慧型手機的價格",
        "獲取耳機的產品詳細資訊和價格",
        "獲取 USB 充電器的詳細資訊",
        "獲取鞋子的產品詳細資訊和價格",
        "獲取喇叭的產品詳細資訊？",
    ],
    "reference_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "smartphone"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "headphones"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "headphones"},
            },
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "usb charger"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "shoes"},
            },
            {"tool_name": "get_product_price", "tool_input": {"product_name": "shoes"}},
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "speaker"},
            }
        ],
    ],
    "predicted_trajectory": [
        [
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "smartphone"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "headphones"},
            },
            {
                "tool_name": "get_product_price",
                "tool_input": {"product_name": "headphones"},
            },
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "usb charger"},
            }
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "shoes"},
            },
            {"tool_name": "get_product_price", "tool_input": {"product_name": "shoes"}},
        ],
        [
            {
                "tool_name": "get_product_details",
                "tool_input": {"product_name": "speaker"},
            }
        ],
    ],
    "response": [
        "500",
        "50",
        "一款超快、輕便的 USB 充電器",
        "100",
        "一款聲控智慧喇叭，可播放音樂、設定鬧鐘並控制智慧家居設備。",
    ],
}

byod_eval_sample_dataset = pd.DataFrame(byod_eval_data)
byod_eval_sample_dataset["predicted_trajectory"] = byod_eval_sample_dataset[
    "predicted_trajectory"
].apply(json.dumps)
byod_eval_sample_dataset["reference_trajectory"] = byod_eval_sample_dataset[
    "reference_trajectory"
].apply(json.dumps)

### 執行評估任務

使用您自己的資料集和最新評估的相同設定來執行新的代理評估。

In [ ]:
EXPERIMENT_RUN_NAME = f"response-over-tools-byod-{get_id()}"

byod_response_eval_tool_task = EvalTask(
    dataset=byod_eval_sample_dataset,
    metrics=response_tool_metrics,
    experiment=EXPERIMENT_NAME,
)

byod_response_eval_tool_result = byod_response_eval_tool_task.evaluate(
    experiment_run_name=EXPERIMENT_RUN_NAME
)

display_eval_report(byod_response_eval_tool_result)

### 視覺化評估結果

視覺化評估結果範例。

In [ ]:
display_dataframe_rows(byod_response_eval_tool_result.metrics_table, num_rows=3)

In [ ]:
display_radar_plot(
    byod_response_eval_tool_result,
    title="代理評估指標",
    metrics=[f"{metric}/mean" for metric in response_tool_metrics],
)

## 清理


In [ ]:
delete_experiment = True

if delete_experiment:
    try:
        experiment = aiplatform.Experiment(EXPERIMENT_NAME)
        experiment.delete(delete_backing_tensorboard_runs=True)
    except Exception as e:
        print(e)